# 準備

In [17]:
from dotenv import load_dotenv
load_dotenv() # .envファイルから環境変数を読み込み(OPENAI_API_KEY変数にAPI Keyが入っている)

True

# LLM Chain

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() # これだけでモデルを初期化できるぽい

In [19]:
# これでモデルと会話することができる
# print(llm.invoke("langsmithはどのようにテストを支援してくれますか？"))

In [20]:
# プロンプトテンプレートを使うこともできる
from langchain_core.prompts import ChatPromptTemplate

# この例だとシステムに「技術ドキュメントライター」というロールを与えている
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは世界レベルの技術ドキュメントライターです。"),
    ("user", "{input}")
])

In [21]:
# 組み合わせてchainをつくる
chain = prompt | llm

In [22]:
# chainもinvokeで会話できる
# chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

In [23]:
# 出力をAIMessageからStringに変える
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [24]:
# chainに加える
chain = prompt | llm | output_parser

In [25]:
# chain.invoke({"input": "langsmithはどのようにテストを支援してくれますか？"})

## Diving Deeper

さらに深く理解するためにたくさんのドキュメントが用意されている。今すぐにはとても読めないので、少しずつ読む。

# Retrieval Chain

In [26]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview") # URLを渡すとその内容を取ってきてくれる

docs = loader.load() # 指定したURLからドキュメントを取得する

In [27]:
from langchain_openai import OpenAIEmbeddings # ベクターストアに格納するためにドキュメントをベクトル化するためのモジュール

embeddings = OpenAIEmbeddings()

# これを使って、今からベクターストアにドキュメントを挿入する

In [28]:
# 簡単のため、シンプルなローカルベクターストアであるFAISSを使う
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter() # チャンクサイズの制限を下回るまで再帰的にテキストを分割するTextSplitter
documents = text_splitter.split_documents(docs) # TextSpritterにかける
vector = FAISS.from_documents(documents, embeddings) # ベクトル化する

In [29]:
documents

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, 

In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain 

prompt = ChatPromptTemplate.from_template("""与えられた文脈のみに基づいて質問に回答してください:
<context>
{context}
</context>

質問: {input}""")

# 質問と取得したドキュメントを受け取り、解答を生成するチェーンを作成する
# create_stuff_documents_chainはドキュメントのリストを受け取ってプロンプトにフォーマットし、すべてをLLMに渡す。
document_chain = create_stuff_documents_chain(llm, prompt)

In [32]:
from langchain_core.documents import Document 

# ドキュメントを自分で渡すことで、チェーンを自分で実行することもできる
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by providing the ability to visualize test results.'

In [33]:
# でも、retrieverを使うことで、質問に最も関連性の高いドキュメントを動的に選択してモデルに渡すことができる
from langchain.chains import create_retrieval_chain # retrieval chainを作るための関数

retriever = vector.as_retriever() # retrieverを作成
retrieval_chain = create_retrieval_chain(retriever, document_chain) # retrieverとドキュメントを受け取るchainからretrieval chainを作成

In [35]:
# これで、retrieval chainを呼び出すことができる
response = retrieval_chain.invoke({"input": "LangSmithはどのようにテストを支援してくれますか?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmithは、テストを支援するためのさまざまな機能を提供しています。具体的には、以下のような機能があります。

1. データセットの作成と実行: LangSmithを使用して、データセットを作成し、それをプロンプトやチェーンに対して実行することができます。データセット上でのプロンプトやチェーンの実行結果を視覚化することで、結果を目視で確認することができます。

2. テスト結果の評価: LangSmithでは、テストランの結果を評価するための評価ツールが提供されています。これらの評価ツールを使用することで、自動評価メトリクスや人手による評価結果を取得できます。自動評価メトリクスは一貫性とスケーラビリティに優れていますが、人手による評価はアプリケーションの品質と信頼性を高めるために重要です。

3. 人手による評価: LangSmithでは、人手によるレビューや注釈付けを容易に行うことができます。注釈付けキューを使用することで、モデルの入力や出力を確認し、フィードバックを追加することができます。これにより、主観的な品質や自動評価の結果に対して人手による確認を行うことができます。

4. テスト結果の分析: LangSmithでは、テストランの結果を分析するための機能も提供されています。テストランの結果にフィードバックを追加し、結果をフィルタリングして分析することができます。また、結果の集計統計情報を表示することも可能です。

これらの機能により、LangSmithはテストプロセスをサポートし、アプリケーションの品質と信頼性を向上させるのに役立ちます。


## Diving Deeper

これも少しずつ読む

# Conversation Retrieval Chain

ここまでやってきたのは1つの質問に答えるchain。でもこれだと一問一答で会話ができない。フォローアップクエスチョンにもこたえられるchainを作るにはどうすればいいだろうか。

In [37]:
# create_retrieval_chain()をまだ使うことができる。でも変更を加える必要がある
# 1. 直近の入力だけでなく、すべての会話の歴史を入力に加える
# 2. 最後のLLM chainも同様にすべての会話の歴史を受け取って出力を生成する

from langchain.chains import create_history_aware_retriever # 歴史を考慮したretriever
from langchain_core.prompts import MessagesPlaceholder # メッセージプレースホルダー (何なのかよくわからないぞ) 

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"), # メッセージプレースホルダの中にchat_historyを入れる
    ("user", "{input}"), # ユーザのインプット
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    # 以上を踏まえて、会話に関係する情報を取得するために検索する検索クエリを生成してください
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt) # 歴史を踏まえた生成を行うchainを生成

In [44]:
from langchain_core.messages import HumanMessage, AIMessage # 人間のメッセージとAIのメッセージ

# 仮の歴史
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか？"), AIMessage(content="もちろんです!")]

# chainに入力して続きを出力
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "どのように支援してくれるのですか"
})

# LLMはクエリを生成するため、LangSmithを使ったテストに関するドキュメントを返す(なんでクエリを生成したらドキュメントが返ってくるのかわかんない)

[Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a virtual environment or open our bash shell and leave them set. The same principle app

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "以下のコンテキストを参照してユーザーの質問に答えてください:\n\n{context}"), # コンテキスト
    MessagesPlaceholder(variable_name="chat_history"), # 会話の歴史
    ("user", "{input}"), # ユーザーの入力
])
document_chain = create_stuff_documents_chain(llm, prompt) # 変更したプロンプトで再びchainを生成

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain) # 歴史も参照できるchainを生成

In [46]:
chat_history = [HumanMessage(content="LangSmithは私のLLMアプリケーションのテストを支援してくれますか?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "どのように支援してくれますか？"
})

{'chat_history': [HumanMessage(content='LangSmithは私のLLMアプリケーションのテストを支援してくれますか?'),
  AIMessage(content='Yes!')],
 'input': 'どのように支援してくれますか？',
 'context': [Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work nee